In [1]:
import os
import re
import sys
from collections import OrderedDict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.stats import spearmanr, pearsonr, stats
from scipy import spatial

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn, optim
import torch.optim as optim

tqdm.pandas()

# For Google Colab
if 'google.colab' in sys.modules:
    # from google.colab import drive
    # drive.mount('/content/drive')
    PATH = os.path.join("")

    !pip install lightning
    !pip install huggingface_hub
else:
    PATH = os.path.join("..", "data", "raw")

# Logging to HuggingFace
from huggingface_hub import login
login()

Get Data

In [2]:
def get_data(subset):

    df_train = pd.read_csv(os.path.join(PATH, 'eng_train.csv'))
    df_train["Split_Text"] = df_train["Text"].apply(lambda x: x.replace("\n", " "))
    df_train['Split_Text'] = df_train['Split_Text'].apply(lambda x: x.split("\r"))
    df_train['Split_Text'] = df_train['Split_Text'].apply(lambda x: [re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in x])

    df_train["sen_1"] = df_train["Split_Text"].apply(lambda x: x[0])
    df_train["sen_2"] = df_train["Split_Text"].apply(lambda x: x[1])
    df_train.drop(["Split_Text"], axis=1, inplace=True)
    display(df_train.head())

    if subset is not None:
        df_train = df_train.sample(n=subset, random_state=42)

    return df_train

Model Definition

In [3]:
import lightning as L

class Classifier(L.LightningModule):
    def __init__(self, model_name):
        super(Classifier, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=1
        )
        self.model.config.pad_token_id = self.model.config.eos_token_id

        self.loss_fn = torch.nn.MSELoss()

        self.val_predictions = []
        self.val_labels = []

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids, attention_mask, labels)
        loss = outputs.loss
        return loss

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids, attention_mask)
        predictions = outputs.logits.squeeze()

        # Append predictions and labels to the lists
        self.val_predictions.append(predictions)
        self.val_labels.append(labels.squeeze())

        return predictions, labels

    def on_test_epoch_end(self):
        # Concatenate predictions and labels at the end of each epoch
        predictions = torch.cat(self.val_predictions)
        labels = torch.cat(self.val_labels)

        mse = self.loss_fn(predictions, labels)
        spearman_corr, _ = spearmanr(
            predictions.cpu().numpy(), labels.cpu().numpy()
        )

        # Log the metrics
        self.log("val_loss", mse, prog_bar=True)
        self.log("val_spearman_corr", spearman_corr, prog_bar=True)

        # Clear the lists for the next epoch
        self.val_predictions = []
        self.val_labels = []

    def configure_optimizers(self):
        self.optimizer = optim.AdamW(self.model.parameters(), lr=5e-5)
        return self.optimizer


def prepare_torch_dataset(df, tokenizer):
    tokenized = tokenizer(
        df["sen_1"].tolist(),
        df["sen_2"].tolist(),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
    )

    labels = torch.tensor(df["Score"].tolist(), dtype=torch.float32).unsqueeze(1)
    return TensorDataset(tokenized["input_ids"], tokenized["attention_mask"], labels)


def run_cv(df, model_name, n_splits=5, epochs=3, batch_size=8, test_run=False):
    if test_run:
        n_splits = 2

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 5-fold cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)


    all_spearman_corrs = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):

        print(f"Fold {fold + 1}/{kf.get_n_splits()}")

        # Initialize the Lightning model
        model = Classifier(model_name)

        # Freeze all the parameters except the last classifier layer
        for name, param in model.named_parameters():
            if name in ["model.model.norm.weight", "model.score.weight", "score.weight"]:
                print(name)
                param.requires_grad = True
            else:
                param.requires_grad = False

        # Initialize the Lightning trainer
        trainer = L.Trainer(
            accelerator="auto", max_epochs=epochs, num_sanity_val_steps=0, fast_dev_run=test_run
        )

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            # model.resize_token_embeddings(len(tokenizer))

        # Split data into train and validation sets
        train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]

        train_data = prepare_torch_dataset(train_df, tokenizer)
        val_data = prepare_torch_dataset(val_df, tokenizer)

        train_dataloader = DataLoader(train_data, batch_size=batch_size, num_workers=2, shuffle=True)
        val_dataloader = DataLoader(val_data, batch_size=batch_size, num_workers=2, shuffle=False)


        # Train & Evaluate the model
        trainer.fit(model, train_dataloader)
        trainer.test(model, val_dataloader)


        # Calculate and print the average Spearman correlation
        average_spearman_corr = trainer.callback_metrics["val_spearman_corr"].mean()
        print(
            f"Average Spearman Correlation for Fold {fold + 1}: {average_spearman_corr}"
        )

        all_spearman_corrs.append(average_spearman_corr)

        break
    # Calculate and print the overall average Spearman correlation
    overall_average_spearman_corr = sum(all_spearman_corrs) / len(all_spearman_corrs)
    print(
        f"Overall Average Spearman Correlation across all folds: {overall_average_spearman_corr}"
    )

    return overall_average_spearman_corr

In [ ]:
df = get_data(subset=None) # Set to None to use the full dataset, set to 100 to prototype

# "HuggingFaceH4/tiny-random-LlamaForSequenceClassification"
# "princeton-nlp/Sheared-LLaMA-1.3B",
# "winglian/Llama-2-3b-hf",
# "perlthoughts/Llama-2-3b-hf"

score = run_cv(
    df,
    model_name= "princeton-nlp/Sheared-LLaMA-1.3B",
    n_splits=5,
    epochs=5,
    batch_size=24,
    test_run = False, # Set to True to run a test on one batch
)

,PairID,Text,Score,sen_1,sen_2
0,ENG-train-0000,"It that happens, just pull the plug.\r\nif tha...",1.0,It that happens just pull the plug,if that ever happens just pull the plug
1,ENG-train-0001,A black dog running through water.\r\nA black ...,1.0,A black dog running through water,A black dog is running through some water
2,ENG-train-0002,I've been searchingthe entire abbey for you.\r...,1.0,I ve been searchingthe entire abbey for you,I m looking for you all over the abbey
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,If he is good looking and has a good personali...,If he s good looking and a good personality h...
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,She does not hate you she is just annoyed with...,She doesn t hate you she is just annoyed


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fold 1/5


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/Sheared-LLaMA-1.3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.model.norm.weight
model.score.weight


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type                           | Params
-----------------------------------------------------------
0 | model   | LlamaForSequenceClassification | 1.3 B 
1 | loss_fn | MSELoss                        | 0     
-----------------------------------------------------------
4.1 K     Trainable params
1.3 B     Non-trainable params
1.3 B     Total params

Training: |          | 0/? [00:00<?, ?it/s]

### Model training and evaluation using CV

- 14GB download
- requires ~ 32GB of RAM to run
- freezed LLAMA2 layers, only classifier training


Run with CUDA GPU acceleration
- XXX with a fast CPU
- XXX min on Colab

Results ~ XXX Spearman correlation